In [19]:
from classy import Class
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import scipy.integrate as sc
#plt.rcParams.update({   "text.usetex": True,})
import scipy.interpolate

In [5]:
kvec = np.logspace(1e-3,1e1,20)#
kvec = [1e-3,1e-2,1e-1,1,1e1]
print(kvec)
Omega_M = 0.3
Omega_R = 7e-5
Omega_lambda = 1 - Omega_M - Omega_R

H0 = 67 / (3*1e5) # i 1/Mpc
cosmo_dict ={'H0':70, 'Omega_b':0.05, 'Omega_cdm':0.25, 'k_output_values':str(kvec).strip('[]'), 
           'output':'tCl, lCl, dTk, vTk, pCl', 'lensing':'yes',}

cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
bg = cosmo.get_background()

pts = cosmo.get_perturbations()['scalar']
#print(bg.keys())
pts0=pts[0]
#print(f"{pts0.keys()=}")
#print( cosmo.Hubble(0))



[0.001, 0.01, 0.1, 1, 10.0]


# Neutrino stuff

$T(a) = T_0 * a^{-1}$

$T_0 = 1.9 k$

In [44]:
# nuetrino stuff

# mass

mv = 0.09e-6 # 1 MeV

# temperature 
# today 
# in kelvin

Tv_k = 1.947
# in MeV
kb = 8.617e-11 # MeV/K

T0 = Tv_k * kb
print(T0)


1.6777299e-10


In [7]:
z = bg["z"]
a = 1/(z+1)
aend = a[-1]
a0 = a[0]

In [42]:
def H(a):
    return H0 * np.sqrt(Omega_lambda + Omega_M /( a **3) + Omega_R /( a**4)) 


def tau_cal(a,y):
    # Vores "tids"-værdi i denne differentialligning er a
    tau = y[0]
    d_tau = 1 / (a ** 2 * H(a))
    return np.array(d_tau)

tau0 = (1/H0) * (Omega_R ** (-0.5)) * a0


sol_tau = sc.solve_ivp(tau_cal,t_span=[a0,aend],y0=[tau0])
tau =sol_tau.y[0]

def a_solve(tau,y):
    a = y[0]
    a_dt = a * a * H(a)
    return [a_dt]

a_solve = sc.solve_ivp(a_solve,t_span=[tau[0],tau[-1]],y0=[a0])
a_tau = a_solve.t
a = a_solve.y[0]
print(a[0],a[-1])
a_interp = scipy.interpolate.interp1d(a_tau,a)
print(a_interp(14550))




9.999999999999987e-15 1.0001652631534843
0.9998270450641846


In [46]:
# neutrino temperatur function

def Tv(a):
    return T0 / a # in MeV


def cs(a):
    return 2.680 * Tv(a) / mv

In [54]:
#a0 = a_class[0]


def cb_solve(tau,y):    
    delta = y[0]
    theta = y[1]
    d_delta = - theta
    d_theta = - H(a_interp(tau)) * theta - 3/2 * H(a)**2 * Omega_M * delta
    return [d_delta,d_theta]



delta0 = pts[0]["delta_cdm"][0]
theta0 = pts[0]["theta_cdm"][0]


params = [delta0,theta0]

t_span = [tau0,tau[-1]]

print(params,t_span)
sol_cb = sc.solve_ivp(cb_solve,t_span = t_span, y0 = params)
#delta = sol_cb.y[0]
#print(delta)



[-1.018531191689055, 2.373358813587738e-05] [5.351769892542055e-09, 14552.495074531013]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.